In [1]:
import sympy
from sympy import S, Symbol, symbols, simplify, nonlinsolve, Matrix, diff, factor
from sympy import log as Log

## Занятие 15
## Математический анализ
## Экстремум функции нескольких переменных

Поиск экстремума ФНП:

1. Находим стационарные точки (все частные производные равны нулю) и точки, в которых производные не существуют

2. Проверяем выполнение достаточного условия экстремума.


### Пример 1

Найти точки, в которых возможен экстремум функции $\ln(-x^2 + 3(y - 5)^2 - 6xy)$

Вычисляем частные производные по $x$ и $y$, решаем систему уравнений:

In [2]:
from sympy.abc import x, y
f = Log(-x**2 + 3*(y - 5)**2 - 6*x*y)
stat_points = nonlinsolve([f.diff(x), f.diff(y)], [x, y])
stat_points

546304


{(-15/4, 5/4)}

### Достаточные условия экстремума 

Пусть $P(a,b)$ - стационарная точка функции $f(x,y)$, т.е. $d\,f(a,b)=0$. Обозначим 
$\Delta=\left|\begin{matrix}f''_{xx}(a,b)&f''_{xy}(a,b)\\
f''_{xy}(a,b)&f''_{yy}(a,b)\end{matrix}\right|$.
Тогда

1) если $\Delta>0$, то функция $f(x,y)$ имеет экстремум в точке $P$,

при $f''_{xx}(a,b)>0$ или $f''_{yy}(a,b)>0$ минимум, 

при $f''_{xx}(a,b)<0$ или $f''_{yy}(a,b)<0$ максимум

2) если $\Delta<0$, то у  функции $f(x,y)$ нет экстремума в точке $P$

3) если $\Delta=0$, то требуется дальнейшее исследование.


### Пример 2.
Проверим выполнение достаточных условий экстремума в стационарной точке Примера 1.

In [3]:
Delta = Matrix([[f.diff(x, 2), f.diff(x, y)], [f.diff(x, y), f.diff(y, 2)]]).det()
x0, y0 = tuple(*stat_points)
Delta.subs({x: x0, y: y0})

-256/16875

Вывод - у функции нет точек экстремума.

### Пример 3.
Найти экстремум функции $u=x+\frac{y^2}{4x}+\frac{z^2}{y}+\frac{2}{z}$, $x,y,z>0$

Вначале найдем стационарные точки

In [4]:
x, y, z = symbols('x y z', positive=True)
u = x + y**2/(4*x) + z**2/y + 2/z
stat_points = nonlinsolve([u.diff(x), u.diff(y), u.diff(z)], [x, y, z]) 
display(*stat_points)

(-1/2, -1, -1)

(1/2, 1, 1)

(-I/2, -I, I)

(I/2, I, -I)

(-sqrt(2)*(1 + I)/4, sqrt(2)*(1 + I)/2, -sqrt(2)/2 + sqrt(2)*I/2)

(sqrt(2)*(1 + I)/4, -sqrt(2)*(1 + I)/2, sqrt(2)/2 - sqrt(2)*I/2)

(sqrt(2)*(-1 + I)/4, sqrt(2)*(1 - I)/2, -sqrt(2)/2 - sqrt(2)*I/2)

(sqrt(2)*(1 - I)/4, sqrt(2)*(-1 + I)/2, sqrt(2)/2 + sqrt(2)*I/2)

Составим список только вещественных точек:

In [5]:
def Delta(u, x, y, z):
    return Matrix([[u.diff(x, 2), u.diff(x, y), u.diff(x, z)], 
                [u.diff(y, x), u.diff(y, 2), u.diff(y, z)],
                [u.diff(z, x), u.diff(z, y), u.diff(z,2)],]).det()
for point in stat_points:
        x0, y0, z0 = point 
        if x0.is_real and y0.is_real and z0.is_real:
            if Delta(u, x, y, z).subs({x: x0, y: y0, z: z0}) > 0:
                A = u.diff(x, 2).subs({x: x0, y: y0, z: z0})
                B = u.diff(y, 2).subs({x: x0, y: y0, z: z0})
                C = u.diff(z, 2).subs({x: x0, y: y0, z: z0})
                if A > 0 or B > 0 or C > 0:
                    display('minimum', point)
                elif A < 0 or B < 0 or C < 0:
                    display('maximum', point)         

'minimum'

(1/2, 1, 1)

Ответ: точка (1/2,1,1) - точка минимума, минимальное значение функции 4.


### Условный экстремум. Метод множителей Лагранжа

Условный экстремум функции $f(x,y)$ - максимум или минимум этой функции при условии, что $x$ и $y$ связаны уравнением связи $\phi(x,y)=0$.

Составим функцию Лагранжа
$$
L(x,y)=f(x,y)+\lambda\phi(x,y)
$$

Функция $f(x,y)$ имеет условный экстремум в стационарных точках функции Лагранжа, если второй дифференциал функции Лагранжа положителен (минимум) или отрицателен (максимум).

### Пример 4. 
Найти экстремум функции 
$f=6-4x-3y$ при условии $x^2+y^2=1$. Физический смысл - найти самую высокую точку кривой, образующейся при пересечении цилиндра $x^2+y^2=1$ плоскостью $f=6-4x-3y$.

Составляем функцию Лагранжа и ищем условный экстремум. Вначале найдем стационарные точки.

In [6]:
x, y, lam = symbols('x y lamda')# нужны символы без ограничений по знаку!
varrs = [x, y, lam]
f = 6 - 4*x - 3*y
restriction = x**2 + y**2 - 1
L = f + lam*restriction
stat_points = nonlinsolve([L.diff(var)for var in varrs], varrs)
display(stat_points, *varrs)

FiniteSet((-4/5, -3/5, -5/2), (4/5, 3/5, 5/2))

x

y

lamda

Теперь составляем выражение для второго дифференциала функции Лагранжа

In [7]:
dx, dy, dx2, dy2 = symbols('dx dy dx^2 dy^2')
d2L = L.diff(x,2)*dx2 + 2*L.diff(x,y)*dx*dy + L.diff(y,2)*dy2
factor(d2L)

2*lamda*(dx^2 + dy^2)

Ясно, что при $\lambda>0$ второй дифференциал положителен, при $\lambda<0$ отрицателен, поэтому точка (-4/5,-3/5) - точка локального максимума,  (4/5,3/5) - точка локального минимума.

Найдем максимальное и минимальное значение функции $f=6-4x-3y$ при условии $x^2+y^2=1$:

In [8]:
ff = Symbol('f')
for point in stat_points:
    display(point[:-1], sympy.Eq(ff, f.subs({var: point[j] for j, var in enumerate(varrs)})))

(-4/5, -3/5)

Eq(f, 11)

(4/5, 3/5)

Eq(f, 1)

### Пример 5.
С помощью метода Лагранжа найти экстремум функции $u = x^2 + y^2 + z^2$ при условии $\frac{x^2}{25} + \frac{y^2}{9} + \frac{z^2}{4} = 1$.

In [9]:
x, y, z, lam = symbols('x y z lamda')
varrs = [x, y, z, lam]
u = x**2 + y**2 + z**2
restriction = x**2/25 + y**2/9 + z**2/4 - 1
L = u + lam*restriction
stat_points = nonlinsolve([L.diff(var)for var in varrs], varrs)
display(stat_points)
dx, dy, dz, dx2, dy2, dz2 = symbols('dx dy dz dx^2 dy^2 dz^2')
d2L = 0
diffs = {x: (dx, dx2), y: (dy, dy2), z: (dz, dz2)}
for i, var in enumerate(varrs[:-1]):
    d2L += L.diff(var, 2)*diffs[var][1]
    for j in range(i + 1, 3):
        d2L += 2*L.diff(var, varrs[j])*diffs[var][0]*diffs[varrs[j]][0]
display(d2L)

FiniteSet((-5, 0, 0, -25), (0, -3, 0, -9), (0, 0, -2, -4), (0, 0, 2, -4), (0, 3, 0, -9), (5, 0, 0, -25))

2*dx^2*(lamda/25 + 1) + 2*dy^2*(lamda/9 + 1) + dz^2*(lamda/2 + 2)

Подставим значения $\lambda$ стационарных точек в выражение для второго дифференциала:

In [10]:
L2_points = [d2L.subs(lam, lval) for lval in (-4, -9, -25)]
display(*L2_points)

42*dx^2/25 + 10*dy^2/9

32*dx^2/25 - 5*dz^2/2

-32*dy^2/9 - 21*dz^2/2

При $\lambda = -4$ второй дифференциал положителен, стационарные точки (0, 0, -2) и (0, 0, 2) - точки минимума.

При $\lambda = -9$ второй дифференциал не является знакопостоянным, соответствующие стационарные точки не являются точками экстремума.

При $\lambda = -25$ второй дифференциал отрицателен, стационарные точки (-5, 0, 0) и (5, 0, 0) - точки максимума.

In [11]:
uu = Symbol('u')
for point in stat_points:
    if point[-1] != -9:
        display(point[:-1], sympy.Eq(uu, u.subs({var: point[j] for j, var in enumerate(varrs)})))

(-5, 0, 0)

Eq(u, 25)

(0, 0, -2)

Eq(u, 4)

(0, 0, 2)

Eq(u, 4)

(5, 0, 0)

Eq(u, 25)